In [1]:
import folium
import requests
import pandas as pd
import numpy as np 
import random
import json
from geopy.geocoders import Nominatim
from IPython.display import Image 
from IPython.core.display import HTML 
from pandas.io.json import json_normalize
import folium
from bs4 import BeautifulSoup
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from random import randint
from sklearn.cluster import DBSCAN 
from sklearn.datasets import make_blobs 
from sklearn.preprocessing import StandardScaler 
pd.options.mode.chained_assignment = None  # default='warn'
print('Libraries imported.')

Libraries imported.


In [2]:
csv='AB_NYC_2019.csv'
df=pd.read_csv(csv)
df.drop(columns=['minimum_nights','reviews_per_month','number_of_reviews','last_review','calculated_host_listings_count','availability_365'],inplace=True)
df.dropna(subset=['latitude','longitude','price'],how='any',inplace=True)

In [3]:
df.describe()

,id,host_id,latitude,longitude,price
count,4.889500e+04,4.889500e+04,48895.000000,48895.000000,48895.000000
mean,1.901714e+07,6.762001e+07,40.728949,-73.952170,152.720687
std,1.098311e+07,7.861097e+07,0.054530,0.046157,240.154170
min,2.539000e+03,2.438000e+03,40.499790,-74.244420,0.000000
25%,9.471945e+06,7.822033e+06,40.690100,-73.983070,69.000000
50%,1.967728e+07,3.079382e+07,40.723070,-73.955680,106.000000
75%,2.915218e+07,1.074344e+08,40.763115,-73.936275,175.000000
max,3.648724e+07,2.743213e+08,40.913060,-73.712990,10000.000000


In [4]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48895 entries, 0 to 48894
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   48895 non-null  int64  
 1   name                 48879 non-null  object 
 2   host_id              48895 non-null  int64  
 3   host_name            48874 non-null  object 
 4   neighbourhood_group  48895 non-null  object 
 5   neighbourhood        48895 non-null  object 
 6   latitude             48895 non-null  float64
 7   longitude            48895 non-null  float64
 8   room_type            48895 non-null  object 
 9   price                48895 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 4.1+ MB


In [6]:
df.dropna()
df.shape

(48895, 10)

In [7]:
# Separate database by Neighbourhood Group. There are 5 groups: Brooklyn, Manhattan, Queens, Staten Island, Bronx
df_Brooklyn=df.loc[df.neighbourhood_group=='Brooklyn']
df_Manhattan=df.loc[df.neighbourhood_group=='Manhattan']
df_Queens=df.loc[df.neighbourhood_group=='Queens']
df_Staten_Island=df.loc[df.neighbourhood_group=='Staten Island']
df_Bronx=df.loc[df.neighbourhood_group=='Bronx']


In [46]:
Brooklyn=[df_Brooklyn.latitude.max(),df_Brooklyn.latitude.min(),df_Brooklyn.longitude.max(),df_Brooklyn.longitude.min()]
Manhattan=[df_Manhattan.latitude.max(),df_Manhattan.latitude.min(),df_Manhattan.longitude.max(),df_Manhattan.longitude.min()]
Queens=[df_Queens.latitude.max(),df_Queens.latitude.min(),df_Queens.longitude.max(),df_Queens.longitude.min()]
Staten_Island=[df_Staten_Island.latitude.max(),df_Staten_Island.latitude.min(),df_Staten_Island.longitude.max(),df_Staten_Island.longitude.min()]
Bronx=[df_Bronx.latitude.max(),df_Bronx.latitude.min(),df_Bronx.longitude.max(),df_Bronx.longitude.min()]

In [8]:

def kmeans_processing(zone):
    # Load and process raw data
    csv='AB_NYC_2019.csv'
    df=pd.read_csv(csv)
    # Drop uneccesary info
    df.drop(columns=['minimum_nights','reviews_per_month','number_of_reviews','last_review','calculated_host_listings_count','availability_365'],inplace=True)
    df.dropna(subset=['latitude','longitude','price'],how='any',inplace=True)
    # Zone filter
    df_=df.loc[df.neighbourhood_group==zone]
    # DBSCAN Processing
    Clus_dataSet=df_[['latitude','longitude']]
    db=DBSCAN(eps=0.003, min_samples=15).fit(Clus_dataSet)
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    # Done processing DB Scan, now add a row to df to filter out outliers of location. 
    df_['label_DB_Scan']=db.labels_
    df_=df_[df_['label_DB_Scan']!=-1]
#     df_.drop(['label_DB_Scan'],inplace=True)
    # Apply Kmeans to find centers of each neighbourhood:
    X=df_[['latitude','longitude']]
    k_means = KMeans(init="k-means++", n_clusters=len(df_.neighbourhood.unique()), n_init=12)
    k_means.fit(X)
    k_means_labels = k_means.labels_
    df_['label']=k_means_labels
    k_means_cluster_centers = k_means.cluster_centers_
    Centers=pd.DataFrame(k_means_cluster_centers,columns=['latitude', 'longitude'])
    Centers['neighbourhood_group']=zone
    return df_, Centers
def draw_map(zone):
    df_=kmeans_processing(zone)[0]
    Centers=kmeans_processing(zone)[1]
    location_lat=df_.describe().latitude.iloc[1]
    location_loc=df_.describe().longitude.iloc[1]
    rainbow=['#%06X' % randint(0, 0xFFFFFF) for i in range(len(df_.label.unique()))]
    
    drmap = folium.Map(location=[location_lat, location_loc], zoom_start=10)

    
    for lat, lng, neighbourhood_group, neighbourhood, labels in zip(df_['latitude'], df_['longitude'],df_['neighbourhood_group'],df_['neighbourhood'],df_['label']):
        label = '{}, {}'.format(neighbourhood_group, neighbourhood)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            color=rainbow[labels-1],
            fill=True,
#             fill_color='#3186cc',
            fill_color=rainbow[labels-1],
            fill_opacity=0.7,
            parse_html=False).add_to(drmap)  
    for lati, lngi in zip(Centers['latitude'],Centers['longitude']):
        folium.CircleMarker(
            [lati, lngi],
            radius=10,
            color='black',
            fill=True,
            fill_color='black',
            fill_opacity=0.7,
            parse_html=False).add_to(drmap)
    return drmap

In [33]:
# We define the Centerpoints of each Neighbour_hood in each Neighbour_hood_group:
Center_points=pd.DataFrame(columns=['latitude','longitude','neighbourhood_group','label'])
for n,zone in enumerate(df.neighbourhood_group.unique()):
    Center_points_temp=pd.DataFrame(kmeans_processing(zone)[1],columns=['latitude','longitude','neighbourhood_group'])
    Center_points_temp['label']=n
    Center_points=pd.concat([Center_points,Center_points_temp],axis=0)
Center_points.sample(10)

,latitude,longitude,neighbourhood_group,label
8,40.731528,-74.002675,Manhattan,1
9,40.745632,-73.950557,Queens,2
2,40.666298,-73.771238,Queens,2
11,40.779392,-73.980691,Manhattan,1
5,40.637598,-73.962849,Brooklyn,0
1,40.725298,-73.981220,Manhattan,1
2,40.820279,-73.942186,Manhattan,1
2,40.812161,-73.916926,Bronx,4
19,40.667864,-73.928329,Brooklyn,0
16,40.655674,-73.959088,Brooklyn,0


In [10]:
Center_points.neighbourhood_group.value_counts()

Brooklyn         38
Manhattan        31
Queens           24
Bronx            13
Staten Island     3
Name: neighbourhood_group, dtype: int64

In [36]:
draw_map=folium.Map(location=[Center_points.iloc[0][0],Center_points.iloc[0][1]], zoom_start=10)
# colors=['#%06X' % randint(0, 0xFFFFFF) for i in range(1000)]
colors=['red','blue','green','yellow','purple']
for lat, long, location,labels in zip(Center_points['latitude'],Center_points['longitude'],Center_points['neighbourhood_group'],Center_points['label']):
    folium.CircleMarker(
        [lat,long],popup=location,
        radius=5,
            color=colors[labels],
            fill=True,
            fill_color=colors[labels],

            fill_opacity=0.7,
            parse_html=False).add_to(draw_map)
draw_map

In [12]:
# Initiate Four Square API
CLIENT_ID = '5M5MTCRHVZZ2BXVS5JQE22GGGAFXF2BUG3SEL0LZRMPHT3MB' # your Foursquare ID
CLIENT_SECRET = 'BRL3SVW451CJF140BTFJKI2BU3YKFPXKBJJM1IJNXEW4233Q' # your Foursquare Secret
ACCESS_TOKEN = 'BJCW15XSCTWTJZKQOWFU33NQFMOBLKK5PYP3UV0EH5GWJBOY' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5M5MTCRHVZZ2BXVS5JQE22GGGAFXF2BUG3SEL0LZRMPHT3MB
CLIENT_SECRET:BRL3SVW451CJF140BTFJKI2BU3YKFPXKBJJM1IJNXEW4233Q


In [13]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
db_df=pd.DataFrame(columns=['name', 'categories', 'address', 'lat', 'lng', 'labeledLatLngs','distance', 'cc', 'city', 'state', 'country', 'formattedAddress', 'id'])

for latitude, longitude, search_query,labels in zip(Center_points['latitude'],Center_points['longitude'],Center_points['neighbourhood_group'],Center_points['label']):
    radius=500
    search_query='Vietnamese'
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
    location=f'{search_query}_{labels}'
    results= requests.get(url).json()

    venues = results['response']['venues']
    dataframe = json_normalize(venues)

    filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
   
    dataframe_filtered = dataframe.loc[:, filtered_columns]
    dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
    dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
    db_df_temp=pd.DataFrame(dataframe_filtered)
    db_df=pd.concat([db_df,db_df_temp],axis=0)


In [27]:
# We save the result to local to enhance the speed
# db_df.to_csv('List_of_Restaurants_in_Center_of_Neighbourhoods_in_NY')

In [16]:
draw_map_Restaurant=folium.Map(location=[Center_points.iloc[0][0],Center_points.iloc[0][1]], zoom_start=10)
colors=['#%06X' % randint(0, 0xFFFFFF) for i in range(1000)]
# for lat, long,labels in zip(db_df['location.lat'],db_df['location.lng'],db_df['borough']):
#     folium.CircleMarker(
#         [lat,long],
#         radius=5,
#             color='blue',
#             fill=True,
#             fill_color='green',

#             fill_opacity=0.7,
#             parse_html=False).add_to(draw_map_Restaurant)
for lat, long, location,labels in zip(Center_points['latitude'],Center_points['longitude'],Center_points['neighbourhood_group'],Center_points['label']):
    folium.CircleMarker(
        [lat,long],popup=location,
        radius=5,
            color='red',
            fill=True,
            fill_color='red',

            fill_opacity=0.7,
            parse_html=False).add_to(draw_map_Restaurant)
draw_map_Restaurant

In [17]:
data_Italian=pd.read_csv('List_of_Italian_Restaurants_in_Center_of_Neighbourhoods_in_NY')

In [18]:
filtered_columns = ['name', 'categories','location.address','location.lat','location.lng','location.distance','location.city'] 
data_Italian=data_Italian[filtered_columns]
data_Italian=data_Italian[data_Italian.categories!='[]']
data_Italian.to_csv('Italian_Restaurants_in_Center_of_Neighbourhoods_in_NY')


In [19]:
data_Italian['type']=data_Italian['categories']
A=pd.Series(data_Italian['type']).to_list()

data_Italian['type']=pd.Series([A[i].split()[6] if A[i].split()[6]!= "'pluralName':" else A[i].split()[7] for i in range(len(A))])


In [21]:
data_Italian=pd.concat([data_Italian.loc[data_Italian.type=="'shortName':"],data_Italian.loc[data_Italian.type=="'Italian"],data_Italian.loc[data_Italian.type=="'Pizza"]],axis=0)

In [69]:
# Now we calculate the minimum distance from the Restaurants to the Centers, to define the neighbourhood_group of each Restaurant
temp_value=[]
temp_location=[]
neighborhood=[]
for lat,lng in zip(data_Italian['location.lat'],data_Italian['location.lng']):
    for lat_c,lng_c,location in zip(Center_points['latitude'],Center_points['longitude'],Center_points['neighbourhood_group']):
        temp_value.append(abs(lat-lat_c)+abs(lng-lng_c))
        temp_location.append(location)
    neighborhood.append(temp_location[temp_value.index(min(temp_value))])
data_Italian['neighborhood_group']=pd.Series(neighborhood)
data_Italian.head()

,name,categories,location.address,location.lat,location.lng,location.distance,location.city,type,neighborhood_group
0,Circo's Pastry Shop,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",312 Knickerbocker Ave,40.701525,-73.923215,207,Brooklyn,'shortName':,Brooklyn
32,Vino Fine Wine & Spirits,"[{'id': '4bf58dd8d48988d119951735', 'name': 'W...",121 E 27th St,40.742358,-73.983710,309,New York,'shortName':,Manhattan
62,Italian Ice Cart,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",W 18th St,40.744268,-73.998099,97,New York,'shortName':,Manhattan
72,Pasticceria Rocco,"[{'id': '4bf58dd8d48988d1d0941735', 'name': 'D...",243 Bleecker St,40.730806,-74.002763,482,New York,'shortName':,Manhattan
87,Montebello,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",120 East 56th Street,40.760544,-73.970579,460,New York,'shortName':,Manhattan


In [76]:
# From the result of previous steps, we can know how many Italian Restaurant in each neighbourhood_group
data_Italian['neighborhood_group']
pd.Series(neighborhood).value_counts()

Manhattan    143
Brooklyn       2
dtype: int64

In [39]:
Italian_Restaurant=folium.Map(location=[Center_points.iloc[0][0],Center_points.iloc[0][1]], zoom_start=10)
colors=['#%06X' % randint(0, 0xFFFFFF) for i in range(1000)]

colors=['red','blue','green','yellow','purple']
for lat, long, location,labels in zip(Center_points['latitude'],Center_points['longitude'],Center_points['neighbourhood_group'],Center_points['label']):
    folium.CircleMarker(
        [lat,long],popup=location,
        radius=5,
            color=colors[labels],
            fill=True,
            fill_color=colors[labels],

            fill_opacity=0.7,
            parse_html=False).add_to(Italian_Restaurant)
for lat, long,labels in zip(data_Italian['location.lat'],data_Italian['location.lng'],data_Italian['location.address']):
    folium.CircleMarker(
        [lat,long],
        radius=5,popup=str(labels),
            color='black',
            fill=True,
            fill_color='white',

            fill_opacity=0.7,
            parse_html=False).add_to(Italian_Restaurant)
Italian_Restaurant

0      312 Knickerbocker Ave
32             121 E 27th St
62                 W 18th St
72           243 Bleecker St
87      120 East 56th Street
               ...          
206               on Ditmars
211         3241 Steinway St
220            2334 30th Ave
221             3002 30th St
224          2372 Arthur Ave
Name: location.address, Length: 145, dtype: object

In [40]:
data_Italian

,name,categories,location.address,location.lat,location.lng,location.distance,location.city,type
0,Circo's Pastry Shop,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",312 Knickerbocker Ave,40.701525,-73.923215,207,Brooklyn,'shortName':
32,Vino Fine Wine & Spirits,"[{'id': '4bf58dd8d48988d119951735', 'name': 'W...",121 E 27th St,40.742358,-73.983710,309,New York,'shortName':
62,Italian Ice Cart,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",W 18th St,40.744268,-73.998099,97,New York,'shortName':
72,Pasticceria Rocco,"[{'id': '4bf58dd8d48988d1d0941735', 'name': 'D...",243 Bleecker St,40.730806,-74.002763,482,New York,'shortName':
87,Montebello,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",120 East 56th Street,40.760544,-73.970579,460,New York,'shortName':
...,...,...,...,...,...,...,...,...
206,Joe's Pork Store,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",on Ditmars,40.773625,-73.907073,273,Astoria,'Pizza
211,Combo Pizza Deli and Italian Restaurant,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",3241 Steinway St,40.757771,-73.919624,513,Astoria,'Pizza
220,Lanterna Italian Restaurant,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",2334 30th Ave,40.768589,-73.925575,213,Astoria,'Pizza
221,Ralph’s Italian Ices,"[{'id': '4bf58dd8d48988d1c9941735', 'name': 'I...",3002 30th St,40.767151,-73.922395,278,Astoria,'Pizza


In [42]:
data_Italian['location.city'].value_counts()

New York            111
Brooklyn             10
Astoria               8
Bronx                 2
Sunnyside             2
Forest Hills          1
Elmhurst              1
Jackson Heights       1
Mountain View         1
Brooklyn Heights      1
Queens                1
Ridgewood             1
Flushing              1
Name: location.city, dtype: int64

In [66]:

temp_value=[]
temp_location=[]
loc=[]
for lat,lng in zip(data_Italian['location.lat'],data_Italian['location.lng']):
    for lat_c,lng_c,location in zip(Center_points['latitude'],Center_points['longitude'],Center_points['neighbourhood_group']):
        temp_value.append(abs(lat-lat_c)+abs(lng-lng_c))
        temp_location.append(location)
    loc.append(temp_location[temp_value.index(min(temp_value))])

145

SyntaxError: invalid syntax (<ipython-input-50-c6996caa05b5>, line 1)

6

4